In [1]:
from vllm import LLM

llm = LLM(
    model='Qwen/Qwen3-VL-Embedding-2B',
    runner='pooling',
    max_model_len=4096,
    gpu_memory_utilization=0.85,
    quantization="mxfp4"
)

INFO 01-23 22:10:48 [utils.py:263] non-default args: {'runner': 'pooling', 'max_model_len': 4096, 'gpu_memory_utilization': 0.85, 'disable_log_stats': True, 'quantization': 'mxfp4', 'model': 'Qwen/Qwen3-VL-Embedding-2B'}
INFO 01-23 22:10:50 [model.py:859] Resolved `--convert auto` to `--convert embed`. Pass the value explicitly to silence this message.
INFO 01-23 22:10:50 [model.py:530] Resolved architecture: Qwen3VLForConditionalGeneration
INFO 01-23 22:10:50 [model.py:1545] Using max model len 4096
INFO 01-23 22:10:50 [scheduler.py:229] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-23 22:10:53 [vllm.py:630] Asynchronous scheduling is enabled.
INFO 01-23 22:10:53 [vllm.py:637] Disabling NCCL for DP synchronization when using async scheduling.
WARNING 01-23 22:10:53 [vllm.py:744] Pooling models do not support full cudagraphs. Overriding cudagraph_mode to PIECEWISE.
(EngineCore_DP0 pid=154930) INFO 01-23 22:10:56 [core.py:97] Initializing a V1 LLM engine (v0.14.0)

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=154930) INFO 01-23 22:11:14 [default_loader.py:291] Loading weights took 2.36 seconds
(EngineCore_DP0 pid=154930) INFO 01-23 22:11:15 [gpu_model_runner.py:3905] Model loading took 4.31 GiB memory and 4.294866 seconds
(EngineCore_DP0 pid=154930) INFO 01-23 22:11:16 [gpu_model_runner.py:4715] Encoder cache will be initialized with a budget of 8192 tokens, and profiled with 2 video items of the maximum feature size.
(EngineCore_DP0 pid=154930) INFO 01-23 22:11:31 [backends.py:644] Using cache directory: /home/administrator/.cache/vllm/torch_compile_cache/b239d35871/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=154930) INFO 01-23 22:11:31 [backends.py:704] Dynamo bytecode transform time: 7.73 s
(EngineCore_DP0 pid=154930) INFO 01-23 22:11:37 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 1.671 s
(EngineCore_DP0 pid=154930) INFO 01-23 22:11:37 [monitor.py:34] torch.compile takes 9.40 s in total
(Eng

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:04<00:00, 10.41it/s]


(EngineCore_DP0 pid=154930) INFO 01-23 22:11:46 [gpu_model_runner.py:4856] Graph capturing finished in 6 secs, took 0.30 GiB
(EngineCore_DP0 pid=154930) INFO 01-23 22:11:47 [core.py:273] init engine (profile, create kv cache, warmup model) took 31.39 seconds
INFO 01-23 22:11:48 [llm.py:347] Supported tasks: ['embed', 'token_embed']


In [2]:
from sqlalchemy import create_engine, text
from e_commerce_recommendation.configs.settings import Settings
import polars as pl
settings = Settings()
DATABASE_URL = settings.DATABASE_URL

engine = create_engine(DATABASE_URL)
categories = pl.read_database(
    query="SELECT * FROM categories",
    connection=engine
)
categories

id,name
i64,str
0,"""groceries"""
1,"""smart home security & lighting"""
2,"""string instruments"""
3,"""computer screws"""
4,"""home use medical supplies & eq…"
…,…
503,"""motorbike seat covers"""
504,"""bedsheets"""
505,"""statement jewelery"""


In [3]:
categories['name'].to_list()

['groceries',
 'smart home security & lighting',
 'string instruments',
 'computer screws',
 'home use medical supplies & equipment',
 'fragrances',
 'smart home',
 'automotive tools equipment',
 "Men's coats, jackets and vests",
 'electrical',
 'beading & jewelry making',
 'bath body',
 'computer memory',
 'consoles',
 "men's fashion",
 'computer audio video accessories',
 'action sports',
 'home decor',
 'cable accessories',
 'camera & photo accessories',
 'indoor lighting',
 'accessories & supplies',
 "men's jewelry",
 'backpacks',
 'collectible toys',
 'garden decor',
 'heating cooling air quality',
 'car & motorbike',
 'ironing & steamers',
 'bathroom furniture',
 'household cleaning',
 'accessories',
 "kids' party supplies",
 'home kitchen',
 'men',
 'garden tools & watering equipment',
 "men's wallets",
 'beauty',
 'boys',
 "kids' play cars & race cars",
 'bathroom linen',
 'computer monitor accessories',
 'computers & tablets',
 'travel duffel bags',
 'exercise fitness equipmen

In [4]:
embedding_outputs = llm.embed(prompts=categories['name'].to_list())

Adding requests:   0%|          | 0/508 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/508 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
vectors_as_lists = [str(output.outputs.embedding) for output in embedding_outputs]

category_embeddings_df = pl.DataFrame(
    {"category_id": categories["id"], "embedding": vectors_as_lists}
)

category_embeddings_df

category_id,embedding
i64,str
0,"""[0.010615784674882889, 0.02539…"
1,"""[0.006268929690122604, -0.0446…"
2,"""[0.025873634964227676, -0.0508…"
3,"""[-0.018973372876644135, -0.025…"
4,"""[-0.016970675438642502, 0.0151…"
…,…
503,"""[-0.05216212943196297, -0.0454…"
504,"""[-0.08463685214519501, -0.0274…"
505,"""[0.018541019409894943, -0.0829…"


In [7]:
category_embeddings_df.write_database(
    table_name="category_embeddings",
    connection=engine,
    if_table_exists="append"
)

508

In [27]:
pl.DataFrame([embedding.outputs.embedding for embedding in embedding_outputs])

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_471,column_472,column_473,column_474,column_475,column_476,column_477,column_478,column_479,column_480,column_481,column_482,column_483,column_484,column_485,column_486,column_487,column_488,column_489,column_490,column_491,column_492,column_493,column_494,column_495,column_496,column_497,column_498,column_499,column_500,column_501,column_502,column_503,column_504,column_505,column_506,column_507
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.010744,0.006176,0.025796,-0.019291,-0.016831,-0.020187,0.004134,0.009687,0.013782,0.018354,0.005908,-0.035199,-0.019965,-0.003527,0.006777,0.005535,-0.006354,0.002216,-0.019259,-0.035729,0.015217,-0.022355,0.054739,-0.075635,0.01943,0.019084,0.029723,-0.022144,-0.008991,0.004199,0.050433,-0.006241,0.002256,0.006978,-0.018011,0.01037,0.020336,…,-0.01022,0.005719,0.008907,-0.032962,0.004951,-0.024585,-0.00536,0.011513,-0.018395,-0.011676,0.028117,-0.009043,-0.010949,-0.00146,-0.001167,-0.039795,-0.028689,0.012945,-0.015934,0.022075,-0.05694,-0.028346,-0.017959,0.007693,0.016914,-0.009267,0.006687,-0.037396,0.028949,-0.002178,0.023552,-0.015501,-0.052284,-0.084848,0.018335,0.024995,-0.008736
0.025526,-0.044761,-0.051297,-0.025194,0.015112,-0.065751,-0.025496,-0.012485,-0.032086,-0.022635,-0.102203,-0.045194,-0.024131,-0.047476,-0.059351,-0.054315,-0.049088,-0.05421,-0.049152,-0.080172,-0.039787,-0.056173,-0.086991,-0.044508,-0.053762,-0.047858,0.014494,-0.033507,-0.034521,-0.001269,0.020989,-0.0928,-0.020814,0.036421,-0.058625,-0.010149,-0.050768,…,-0.026239,-0.019692,-0.008691,-0.066214,-0.039315,-0.056872,-0.048097,-0.04576,-0.037828,-0.045543,-0.026369,-0.032699,-0.037705,-0.04038,-0.007324,-0.026078,-0.006262,-0.089097,-0.057707,0.021931,-0.027743,-0.030527,-0.064537,-0.107039,0.005339,-0.050203,-0.057835,-0.044582,-0.06667,-0.019139,-0.064153,-0.027288,-0.045566,-0.027399,-0.082652,-0.006174,-0.056437
-0.054225,0.040111,0.028596,-0.025914,0.009167,0.011752,0.040793,-0.011265,0.053671,0.006928,0.022892,0.022307,-0.028152,-0.016348,0.02297,-0.022287,0.034275,0.01553,-0.022277,-0.011038,-0.002617,0.011321,0.007286,-0.010691,-0.031118,-0.000048,0.001582,0.026369,0.066443,0.028627,0.020406,0.012701,-0.011644,-0.0153,0.001622,-0.016106,0.012043,…,0.016961,0.038805,0.056887,0.021397,0.027934,0.028732,-0.042592,0.024493,0.001483,0.005403,0.015297,0.021992,0.017257,0.017378,0.023795,0.015602,-0.034951,0.026758,-0.000185,-0.010677,0.049096,-0.006178,0.037233,-0.010407,-0.021644,-0.002271,0.01764,-0.022878,0.01272,-0.051989,0.009825,0.021716,0.011684,0.012963,0.001155,0.020206,0.010162
-0.031006,-0.004487,-0.032282,-0.019003,-0.008272,0.047872,-0.002167,0.011696,-0.012251,-0.021619,-0.030129,-0.005287,-0.023556,-0.002855,-0.003032,0.010701,-0.014015,-0.006007,-0.015737,0.006027,-0.014994,-0.023071,-0.034027,0.006327,0.01797,-0.044041,-0.01957,-0.007648,0.009461,0.014825,0.000148,-0.022172,-0.007277,-0.024181,-0.028146,-0.018166,0.026394,…,0.022035,0.016145,-0.008691,0.019806,-0.010568,-0.036138,0.010213,-0.044,-0.005971,-0.009029,-0.00834,0.000288,-0.019143,0.011465,-0.066924,-0.006746,0.023447,-0.03182,-0.01335,-0.03102,0.017358,-0.019624,-0.015988,0.002063,-0.036982,-0.01547,-0.033833,-0.056314,-0.026902,0.010355,0.045948,0.004465,0.002172,-0.018634,-0.019644,-0.007259,0.006397
0.025382,-0.024706,0.011424,0.028361,0.023778,0.008688,-0.006993,0.003408,-0.023627,0.007255,0.008234,-0.0094